# HQQ —— 半二次量化

半二次量化（Half-Quadratic Quantization, HQQ）通过快速稳健的优化实现了即时量化。它不需要校准数据，并且可以用于量化任何模型。更多详细信息请参阅 [官方包](https://github.com/mobiusml/hqq/)。

### 安装

我们建议您使用以下方法安装最新版本并构建相应的 CUDA 内核：


In [ ]:
pip install hqq


### 量化模型

要量化一个模型，您需要创建一个 [HqqConfig](/docs/transformers/v4.47.0/en/main_classes/quantization#transformers.HqqConfig)。有以下两种方法：

#### 方法 1：所有线性层使用相同的量化配置


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, HqqConfig

# 所有线性层使用相同的量化配置
quant_config = HqqConfig(nbits=8, group_size=64)


#### 方法 2：具有相同标签的每个线性层使用专门的量化配置


In [ ]:
# 不同线性层使用不同的量化配置
q4_config = {'nbits': 4, 'group_size': 64}
q3_config = {'nbits': 3, 'group_size': 32}
quant_config = HqqConfig(dynamic_config={
  'self_attn.q_proj': q4_config,
  'self_attn.k_proj': q4_config,
  'self_attn.v_proj': q4_config,
  'self_attn.o_proj': q4_config,

  'mlp.gate_proj': q3_config,
  'mlp.up_proj': q3_config,
  'mlp.down_proj': q3_config,
})


第二种方法特别适用于量化混合专家模型（Mixture-of-Experts, MoEs），因为专家在较低的量化设置下受影响较小。

然后，您可以按以下方式量化模型：


In [ ]:
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id, 
    torch_dtype=torch.float16, 
    device_map="cuda", 
    quantization_config=quant_config
)


## 优化运行时

HQQ 支持多种后端，包括纯 PyTorch 和自定义反量化 CUDA 内核。这些后端适合旧 GPU 和 peft/QLoRA 训练。为了更快的推理，HQQ 支持 4 位融合内核（TorchAO 和 Marlin），在单个 4090 GPU 上可以达到每秒 200 个 token 的速度。有关如何使用这些后端的更多详细信息，请参阅 [HQQ 文档](https://github.com/mobiusml/hqq/?tab=readme-ov-file#backend)。